In [1]:
# T_final = parse(Int64, ENV["SLURM_ARRAY_TASK_ID"])

using ITensors, ITensorMPS
using JLD2

# ---------- utilities ----------
# |(1,1)⟩ on every site (bond dim 1)
function ones_mps(sites::Vector{<:Index})
    m = MPS(sites, i -> "+")
    s2 = sqrt(2.0)
    @inbounds for i in 1:length(sites)
        m[i] *= s2                      # (1/√2,1/√2) → (1,1)
    end
    return m
end

# Multiply (1,1) on each site in trace_sites (keeps chain length)
function marginalize_inplace!(ψ::MPS, trace_sites; cutoff=1e-10, maxdim=10000)
    sites = siteinds(ψ)
    Kloc = [1.0 1.0; 1.0 1.0]
    for i in sort(unique(trace_sites))
        orthogonalize!(ψ, i)
        Ki = op(Kloc, sites[i])
        ψ = apply(Ki, ψ; cutoff=cutoff, maxdim=maxdim)
    end
    return ψ
end

# Bond-dim-1 MPO projecting specified sites to 0/1; identity elsewhere
function projector_mpo(sites::Vector{<:Index}, assignments::Dict{Int,Int})
    P0 = [1.0 0.0; 0.0 0.0]; P1 = [0.0 0.0; 0.0 1.0]
    ops = Vector{ITensor}(undef, length(sites))
    @inbounds for i in eachindex(sites)
        if haskey(assignments, i)
            ops[i] = assignments[i] == 0 ? op(P0, sites[i]) : op(P1, sites[i])
        else
            ops[i] = op("Id", sites[i])
        end
    end
    return MPO(ops)
end

# Probability of a (partial) assignment; uses Apply (clamped to [0,1])
function prob_of(ψ::MPS, allones::MPS, assign::Dict{Int,Int})
    Π   = projector_mpo(siteinds(ψ), assign)
    num = inner(allones, Apply(Π, ψ))   # ⟨(1,1)^N| Π |ψ⟩
    den = inner(allones, ψ)             # ≈ 1 if you normalize per sweep
    p = (den == 0 ? num : num/den)
    return clamp(p, 0.0, 1.0)
end

# Shannon entropy (nats) of any array; ignores 0·log 0
function shannon_entropy(P::AbstractArray{<:Real})
    H = 0.0
    @inbounds for p in P
        if p > 0.0
            H -= p*log(p)
        end
    end
    return H
end

assign_empty!(d::Dict) = (empty!(d); d)

# EXACT I(A:C | B) by enumerating all bitstrings on [A..C]
function exact_cmi_AC_given_B(ψ::MPS, A::Int, Bsites::AbstractVector{Int}, C::Int)
    @assert A < C
    keep = collect(A:C)
    L = length(siteinds(ψ))
    @assert all(1 .≤ keep .≤ L)
    ψw = copy(ψ)
    if A > 1
        marginalize_inplace!(ψw, 1:(A-1))
    end
    if C < L
        marginalize_inplace!(ψw, (C+1):L)
    end

    sites   = siteinds(ψw)
    allones = ones_mps(sites)

    K  = length(keep)
    BK = length(Bsites)
    @assert K == BK + 2  # kept = A, B..., C

    # map B site -> bit position 0..BK-1
    bpos = Dict{Int,Int}()
    for (j, s) in enumerate(Bsites)
        bpos[s] = j-1
    end

    P = zeros(Float64, 2, 1<<BK, 2)  # (A, Bbits, C)

    assign = Dict{Int,Int}()
    @inbounds for mask in 0:((1<<K)-1)
        assign_empty!(assign)
        a_bit = 0; c_bit = 0; b_index = 0
        for (j, s) in enumerate(keep)
            bit = (mask >> (j-1)) & 0x1
            assign[s] = bit
            if s == A
                a_bit = bit
            elseif s == C
                c_bit = bit
            else
                b_index |= (bit << bpos[s])
            end
        end
        p = prob_of(ψw, allones, assign)
        P[a_bit+1, b_index+1, c_bit+1] = p
    end

    sP = sum(P)
    if sP > 0
        P ./= sP
    end

    H_ABC = shannon_entropy(P)
    PAB   = reshape(sum(P, dims=3),  :, )  # 2 × 2^BK
    PBC   = reshape(sum(P, dims=1),  :, )  # 2^BK × 2
    PB    = sum(P, dims=(1,3))[:]          # 2^BK

    H_AB  = shannon_entropy(PAB)
    H_BC  = shannon_entropy(PBC)
    H_B   = shannon_entropy(PB)

    I_AC_given_B = H_AB + H_BC - H_B - H_ABC
    return I_AC_given_B
end

# evolution helpers
function build_gbond(sites::Vector{<:Index}, h::AbstractMatrix, τ::Real)
    @assert size(h) == (4,4)
    U = exp(τ * h / 2)
    return [op(U, sites[j], sites[j+1]) for j in 1:length(sites)-1]
end

function sweep!(ψ::MPS, Gbond; cutoff=1e-9, maxdim=400, allones::MPS)
    for j in 1:length(Gbond)
        ψ = apply(Gbond[j], ψ; cutoff=cutoff, maxdim=maxdim)
    end
    for j in length(Gbond):-1:1
        ψ = apply(Gbond[j], ψ; cutoff=cutoff, maxdim=maxdim)
    end
    s = inner(allones, ψ)
    if isfinite(s) && s != 0.0
        ψ /= s
    end
    return ψ
end

# # ---------- experiment params ----------
# for _ in 1:1
#     L = 100                     # even
#     @assert iseven(L)
#     sites = siteinds("Qubit", L)
    
#     ψ = MPS(sites, i -> isodd(i) ? "0" : "1")
    
#     h = [0 0 0 0;
#          0 -1 1 0;
#          0  1 -1 0;
#          0 0 0 0]
#     τ = 0.01
#     T = 10 * T_final
#     r_max_req = 10
    
#     midL = L ÷ 2
#     midR = midL + 1
#     rmax_geom = min(midL - 1, L - midR)
#     r_vals = collect(1:min(r_max_req, rmax_geom))  # fit uses r
#     x_vals_plot = 2 .* r_vals                       # plot vs 2r
    
#     Gbond   = build_gbond(sites, h, τ)
#     allones = ones_mps(sites)
    
    
#     for t in 1:T
#         ψ = sweep!(ψ, Gbond; cutoff=1e-9, maxdim=400, allones=allones)
#     end
    
#     ψs = copy(ψ)
#     y_exact = Float64[]
#     for r in r_vals
#         A = midL - r
#         C = midR + r
#         Bsites = (A+1):(C-1)
#         push!(y_exact, exact_cmi_AC_given_B(ψs, A, Bsites, C))
#     end
    
    
#     name = "/scratch/gpfs/hh2288/swSSB/exact_CMI/r=$(2*r_max_req)_t=$(round(τ*T, digits=3)).jld2"
#     info = "L=$(L), τ = $(τ), cutoff=1e-9, maxdim=400"
#     save_object(name, (info, y_exact));
# end



sweep! (generic function with 1 method)

In [2]:
using LinearAlgebra
using ITensors, ITensorMPS
using Plots
using JLD2

In [3]:
include("../src/states.jl")

include("../src/circuits.jl")
include("../src/defs.jl")
include("../src/dynamics.jl")
include("../src/observables.jl")
include("../src/measurements.jl")


forced_measure_with_val (generic function with 2 methods)

In [4]:
L = 10
T = 1
γ = 0.005
λ = 0.0
ψ0 = diagonal_neelstate(L)

my_ψ, data, terminal_data = circuit(ψ0, L, T, γ, λ; observables=Symbol[:exact_CMI_corr], PBC=false, cutoff=1E-8, maxdim=200, r=1);

MethodError: MethodError: no method matching update_data(::DiagonalState, ::Vector{Symbol}, ::Dict{Symbol, Vector}, ::Int64; r::Int64)
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  update_data(::DiagonalState, ::Vector{Symbol}, ::Dict{Symbol, Vector}, ::Int64; rs) got unsupported keyword argument "r"
   @ Main ~/Code/U1_SWSSB/src/circuits.jl:36


In [ ]:
data

In [ ]:
data

In [ ]:
r = 5
midL = L ÷ 2
midR = midL + 1

A = midL - r
C = midR + r
Bsites = (A+1):(C-1)

exact_cmi_AC_given_B(ψ.mps, A, Bsites, C)

In [ ]:
exact_CMI(ψ, A, C)

In [13]:

    L = 100                 # even
    @assert iseven(L)
    sites = siteinds("Qubit", L)
    
    ψ = MPS(sites, i -> isodd(i) ? "0" : "1")
    
    h = [0 0 0 0;
         0 -1 1 0;
         0  1 -1 0;
         0 0 0 0]
    τ = 0.01
    # T = 10 * T_final
    T = 1
    r_max_req = 10
    
    midL = L ÷ 2 # 50
    midR = midL + 1 # 51
    rmax_geom = min(midL - 1, L - midR)
    r_vals = collect(1:min(r_max_req, rmax_geom))  # fit uses r
    x_vals_plot = 2 .* r_vals                       # plot vs 2r
    
    Gbond   = build_gbond(sites, h, τ)
    allones = ones_mps(sites)
    
    
    for t in 1:T
        ψ = sweep!(ψ, Gbond; cutoff=1e-9, maxdim=400, allones=allones)
    end
    
    ψs = copy(ψ)
    y_exact = Float64[]
    for r in [3]
        A = midL - r # 45
        C = midR + r  # 56
        Bsites = (A+1):(C-1) # 46, 47, ..., 55
        push!(y_exact, exact_cmi_AC_given_B(ψs, A, Bsites, C))
        println("r=$r: ", exact_CMI(DiagonalState(ψ), A, C))
    end

r=3: 2.762131579014948e-8 + 0.0im


In [ ]:
to_vector(ψ)

In [ ]:
sum(abs.(to_vector(my_ψ.mps) .- to_vector(ψ)))